In [1]:
import scipy.stats as scs
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy.stats import norm
from random import randint

## Функция для расчета продолжительности эксперимента

In [2]:
def calc_duration_ab_test(bcr, uplift, power=0.8, aplpha=0.05, ratio=0.5, cnt_sided=2):

    # параметры для расчета:
    # - bcr       - текущий показатель конверсии
    # - uplift    - жеалемое увеличение конверсии
    # - power     - значение мощности, по-умолчанию 0.8
    # - pvalue    - значение pvalue, по-умолчанию 0.05
    # - ratio     - соотношение трафика, если 50%/50%, то 0.5, 90%/10%, то 0.9, и т.д.
    # - cnt_sided - односторонний или двусторонной тест, значения 1 и 2, по-умолчанию 2

    mde = bcr + bcr * uplift
    standard_norm = scs.norm(0, 1)
    Z_beta = standard_norm.ppf(power)
    Z_alpha = standard_norm.ppf(1 - aplpha / cnt_sided)

    n_users = (((Z_beta + Z_alpha)**2) * bcr * (1-bcr)) / ((mde-bcr)**2 * ratio * (1-ratio))

    return round(n_users + 1)

# пример
calc_duration_ab_test (0.3, 0.5)

294

## Функция для расчета p-value Хи-квадрат

In [3]:
def calc_pvalue_chi_square_ab_test(users_a, goal_a, users_b, goal_b, cnt_sided=2):
    
    # вариант a - тестовый
    
    # аргументы функции:
    # - users_a     - количество измерений варианте a
    # - goal_a      - количество целевых действий в варианте a
    # - users_b     - количество измерений варианте b
    # - goal_b      - количество целевых действий в варианте b
    # - cnt_sided   - односторонний или двусторонной тест, значения 1 и 2, по-умолчанию двусторонний
    
    # считаем pvalue
    rate_a = goal_a / users_a
    rate_b = goal_b / users_b
    std_a = np.sqrt(rate_a * (1 - rate_a) / users_a)
    std_b = np.sqrt(rate_b * (1 - rate_b) / users_b)
    
    z_score = (rate_b - rate_a) / np.sqrt(std_a**2 + std_b**2)
    pvalue = round(norm.sf(abs(z_score)) * cnt_sided, 4)
    
    if pvalue > 0.5: pvalue = 1 - pvalue
        
    # считаем достаточность трафика
    uplift = rate_b / rate_a - 1
    ratio = max(users_a, users_b) / ((max(users_a, users_b) + min(users_b, 150)))
    
    duration_ab_test = calc_duration_ab_test(rate_a, uplift, ratio=ratio)
    
    if pvalue < 0.05 and duration_ab_test >= (users_a + users_b):
        print('''Pvalue = {pvalue}. Для данного эксперимента необходимо {calc_duration} пользователей, а получено {fact_duration}, этого количества недостаточно, эксперимент остановлен преждевременно 😓😓😓'''. \
        format(pvalue=pvalue, calc_duration=duration_ab_test, fact_duration=users_a + users_b))
    
    elif pvalue < 0.05 and duration_ab_test < (users_a + users_b):
        print('''Pvalue = {pvalue}. Для данного эксперимента необходимо {calc_duration} пользователей, получено {fact_duration}, этого количества достаточно 😎🎈🎁✅'''. \
        format(pvalue=pvalue, calc_duration=duration_ab_test, fact_duration=users_a + users_b))
        
    return pvalue

# пример
users_a = 150
users_b = 150
goal_a = 45
goal_b = 68

calc_pvalue_chi_square_ab_test(users_a, goal_a, users_b, goal_b)

Pvalue = 0.0055. Для данного эксперимента необходимо 281 пользователей, получено 300, этого количества достаточно 😎🎈🎁✅


0.0055

## Функция для расчета p-value Т-тест

In [4]:
def calc_ttest_ab_test(data_control, data_test):
    
    # аргументы функции:
    # - data_control - список значений контрольного варианта
    # - data_test    - список значений тестового варианта
    
    pvalue = ttest_ind(data_control, data_test)[1]
    return pvalue

# пример
data_ab = pd.read_csv('data_ab.csv')
data_test = data_ab[data_ab['group'] == 'test']['value']
data_control = data_ab[data_ab['group'] == 'control']['value']

calc_ttest_ab_test(data_control, data_test)

0.04351509912450062

## Функция для расчета p-value Т-тест с оценкой дисперсии дельта-методом
Подходит для метрик-отнощшений с неуникальными наблюдениями, например, CTR

In [8]:
# сгенеированный датасет
impression_control = [randint(1,600) for i in range(10000)]
click_control = [randint(0,30) for i in range(10000)]

impression_test = [randint(1,600) for i in range(10000)]
click_test = [randint(0,31) for i in range(10000)]

df = pd.DataFrame({
    'impression_control': impression_control,
    'click_control': click_control,
    'impression_test': impression_test,
    'click_test': click_test
})


def var_ratio(click, impression):
    
    # функция рассчтитывает дисперсию дельта-методом
    # аргументы функции:
    # - click - колонка с кликами
    # - impression - колонка с показами
    
    mean_click = np.mean(click)
    mean_impression = np.mean(impression)
    
    var_click = np.var(click,ddof=1)
    var_impression = np.var(impression,ddof=1)
    
    cov_click_impression = np.cov(click,impression,ddof=1)[0][1]
    
    result = (var_click/mean_click**2 + \
              var_impression/mean_impression**2 - \
              2*cov_click_impression/(mean_click*mean_impression))*(mean_click*mean_click)/(mean_impression*mean_impression*len(click))
    
    return result

def calc_ttest_ab_test_delta(df):
    
    # функция рассчтитывает pvalue критерием t-test
    # аргументы функции:
    # - df - датафрейм с колонками: click_control, impression_control, click_test, impression_test
    
    var_control = var_ratio(df['click_control'],df['impression_control'])
    var_treatment = var_ratio(df['click_test'],df['impression_test'])

    mean_control = df['click_control'].sum()/df['impression_control'].sum()
    mean_treatment = df['click_test'].sum()/df['impression_test'].sum()
    
    diff = mean_treatment - mean_control
    var = var_control+var_treatment
    stde = 1.96*np.sqrt(var)
    lower = diff - stde 
    upper = diff + stde
    z = diff/np.sqrt(var)
    pvalue = stats.norm.sf(abs(z))

    return pvalue

calc_ttest_ab_test_delta(df)

0.029395665911783577

## Функция для расчета p-value Манна-Уитни

In [6]:
def calc_mannwhitneyu_ab_test(data_control, data_test):
    
    # аргументы функции:
    # - data_control  - список значений контрольного варианта
    # - data_test     - список значений тестового варианта
    
    pvalue = mannwhitneyu(data_test, data_control)[1]
    return pvalue

# пример
data_ab = pd.read_csv('data_ab.csv')
data_test = data_ab[data_ab['group'] == 'test']['value']
data_control = data_ab[data_ab['group'] == 'control']['value']

calc_mannwhitneyu_ab_test(data_control, data_test)

0.023891722890583424